# Applied Data Science Capstone Project Week 3 Part 3

In [2]:
!pip install geopy

     |████████████████████████████████| 104 kB 6.0 MB/s eta 0:00:01


In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


#### Read csv file created in Part 2

In [5]:
df=pd.read_csv('df_n.csv',index_col=0)
print(df.shape)
#df.reset_index(drop=True, inplace=True)
df.head()

(103, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.752420,-79.329242
1,M4A,North York,Victoria Village,43.730600,-79.313265
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.650295,-79.359166
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.723270,-79.451286
4,M7A,"Queen's Park , Ontario Provincial Government","Queen's Park , Ontario Provincial Government",43.661150,-79.391715


In [6]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)

The dataframe has 15 boroughs and 103 neighborhoods.


#### For illustration purposes, only boroughs that contain the word Toronto is analyzed. Dataframe is sliced from the original dataframe and a new dataframe of the Toronto data is created. 

In [7]:
toronto_df = df[df['Borough'] .str.contains('Toronto')].reset_index(drop=True)
print(toronto_df.shape)
toronto_df.head()

(39, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.650295,-79.359166
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657363,-79.378180
2,M5C,Downtown Toronto,St. James Town,43.651210,-79.375481
3,M4E,East Toronto,The Beaches,43.676531,-79.295425
4,M5E,Downtown Toronto,Berczy Park,43.645160,-79.373675


#### Geographical coordinates of Toronto 

In [8]:
address = 'Toronto, T'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


#### Visualize Toronto and it's neighborhoods in it.

In [9]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [10]:
CLIENT_ID = 'WZYHPIOVUBN43NTSTXNFCCLI5LEYFSJXFV2WVJYB4M0DBQTO' # your Foursquare ID
CLIENT_SECRET = 'FS0BYDCWHTTXT1XQORXAAWFU0OWNSD03YVHBAQSAP0CHH4YY' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: WZYHPIOVUBN43NTSTXNFCCLI5LEYFSJXFV2WVJYB4M0DBQTO
CLIENT_SECRET:FS0BYDCWHTTXT1XQORXAAWFU0OWNSD03YVHBAQSAP0CHH4YY


#### Exploring the first neighborhood

In [11]:
toronto_df.loc[0, 'Neighborhood']

'Regent Park , Harbourfront'

In [12]:
#Get the neighborhood's latitude and longitude values.
neighborhood_latitude = toronto_df.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_df.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_df.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park , Harbourfront are 43.65029500000003, -79.35916572299999.


#### Top 100 venues that are in Regent Park within a radius of 500 meters

In [13]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=WZYHPIOVUBN43NTSTXNFCCLI5LEYFSJXFV2WVJYB4M0DBQTO&client_secret=FS0BYDCWHTTXT1XQORXAAWFU0OWNSD03YVHBAQSAP0CHH4YY&v=20180605&ll=43.65029500000003,-79.35916572299999&radius=500&limit=100'

In [14]:
#Send the GET request and examine the resutls
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e7b263f14a126001b7c83d4'},
 'response': {'headerLocation': 'The Distillery District',
  'headerFullLocation': 'The Distillery District, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 40,
  'suggestedBounds': {'ne': {'lat': 43.65479500450003,
    'lng': -79.35295813298289},
   'sw': {'lat': 43.645794995500026, 'lng': -79.36537331301709}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4ad4c05ef964a520bff620e3',
       'name': 'The Distillery Historic District',
       'location': {'address': 'btwn Front, Cherry, Gardiner & Parliament',
        'lat': 43.65024435658077,
        'lng': -79.35932278633118,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.65024435658077,
          'lng': -79

#### We know that all the information is in the items key. Before we proceed, let's borrow the get_category_type function from the Foursquare lab.

In [15]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [16]:
#clean JSON and strucutre to Pandas

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,name,categories,lat,lng
0,The Distillery Historic District,Historic Site,43.650244,-79.359323
1,Arvo,Coffee Shop,43.649963,-79.361442
2,Distillery Sunday Market,Farmers Market,43.650075,-79.361832
3,Cacao 70,Dessert Shop,43.650067,-79.360723
4,SOMA chocolatemaker,Chocolate Shop,43.650622,-79.358127


#### Number of venues returned by FourSquare

In [17]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

40 venues were returned by Foursquare.


#### Let's create a function to repeat the same process to all the neighborhoods in Toronto

In [18]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [19]:
toronto_venues = getNearbyVenues(names=toronto_df['Neighborhood'],
                                   latitudes=toronto_df['Latitude'],
                                   longitudes=toronto_df['Longitude']
                                  )

Regent Park , Harbourfront
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond , Adelaide , King
Dufferin , Dovercourt Village
The Danforth East
Harbourfront East , Union Station , Toronto Islands
Little Portugal , Trinity
The Danforth West , Riverdale
Toronto Dominion Centre , Design Exchange
Brockton , Parkdale Village , Exhibition Place
India Bazaar , The Beaches West
Commerce Court , Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park , The Junction South
North Toronto West
The Annex , North Midtown , Yorkville
Parkdale , Roncesvalles
Davisville
University of Toronto , Harbord
Runnymede , Swansea
Moore Park , Summerhill East
Kensington Market , Chinatown , Grange Park
Summerhill West , Rathnelly , South Hill , Forest Hill SE , Deer Park
CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport
Rosedale
Enclave of M5E
St. James

#### Size of the resulting dataframe 

In [20]:
print(toronto_venues.shape)
toronto_venues.head()

(1749, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park , Harbourfront",43.650295,-79.359166,The Distillery Historic District,43.650244,-79.359323,Historic Site
1,"Regent Park , Harbourfront",43.650295,-79.359166,Arvo,43.649963,-79.361442,Coffee Shop
2,"Regent Park , Harbourfront",43.650295,-79.359166,Distillery Sunday Market,43.650075,-79.361832,Farmers Market
3,"Regent Park , Harbourfront",43.650295,-79.359166,Cacao 70,43.650067,-79.360723,Dessert Shop
4,"Regent Park , Harbourfront",43.650295,-79.359166,SOMA chocolatemaker,43.650622,-79.358127,Chocolate Shop


#### Let's check how many venues were returned for each neighborhood

In [21]:

toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,61,61,61,61,61,61
"Brockton , Parkdale Village , Exhibition Place",70,70,70,70,70,70
"CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport",71,71,71,71,71,71
Central Bay Street,96,96,96,96,96,96
Christie,11,11,11,11,11,11
Church and Wellesley,88,88,88,88,88,88
"Commerce Court , Victoria Hotel",100,100,100,100,100,100
Davisville,26,26,26,26,26,26
Davisville North,7,7,7,7,7,7


In [22]:
#unique catogeries
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 225 uniques categories.


## Analyze each Neighborhood

In [23]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Poutine Place,Pub,Ramen Restaurant,Record Shop,Residential Building (Apartment / Condo),Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soup Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park , Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park , Harbourfront",0,0,0,0,0,0,0

In [24]:
#dataframe size
toronto_onehot.shape

(1749, 225)

#### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [25]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Poutine Place,Pub,Ramen Restaurant,Record Shop,Residential Building (Apartment / Condo),Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soup Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.016393,0.000000,0.000000,0.000000,0.000000,0.016393,0.000000,0.016393,0.032787,0.000000,0.000000,0.016393,0.032787,0.000000,0.000000,0.016393,0.00,0.000000,0.000000,0.000000,0.032787,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.016393,0.032787,0.00,0.000000,0.000000,0.032787,0.000000,0.000,0.00,0.000000,0.000000,0.032787,0.081967,0.000000,0.000000,0.00,0.00,0.016393,0.000000,0.016393,0.000000,0.016393,0.016393,0.000000,0.000000,0.000000,0.000000,0.016393,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.016393,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.032787,0.000000,0.0,0.000000,0.016393,0.000000,0.000000,0.000000,0.000000,0.00,0.016393,0.016393,0.016393,0.00,0.00000,0.000000,0.000000,0.000000,0.00,0.000000,0.0000

In [26]:
#size
toronto_grouped.shape

(39, 225)

#### Top 5 most common venue

In [27]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.08
1          Restaurant  0.05
2  Seafood Restaurant  0.05
3               Hotel  0.03
4         Cheese Shop  0.03


----Brockton , Parkdale Village , Exhibition Place----
                    venue  freq
0             Coffee Shop  0.10
1              Restaurant  0.07
2  Furniture / Home Store  0.06
3                    Café  0.06
4                  Bakery  0.04


----CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport----
                venue  freq
0         Coffee Shop  0.08
1  Italian Restaurant  0.07
2          Restaurant  0.06
3                Café  0.04
4                 Bar  0.04


----Central Bay Street----
                       venue  freq
0                Coffee Shop  0.12
1             Clothing Store  0.07
2  Middle Eastern Restaurant  0.03
3         Italian Restaurant  0.02
4           Sushi Restaurant  0.02


----Christie----
              

#### Lets put that into a dataframe

In [46]:
#function to sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [50]:
#Now let's create the new dataframe and display the top 10 venues for each neighborhood.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Restaurant,Seafood Restaurant,Farmers Market,Café,Bakery,Hotel,Cocktail Bar,Beer Bar,Cheese Shop
1,"Brockton , Parkdale Village , Exhibition Place",Coffee Shop,Restaurant,Café,Furniture / Home Store,Bakery,Bar,Vegetarian / Vegan Restaurant,Supermarket,Gym,Sandwich Place
2,"CN Tower , King and Spadina , Railway Lands , ...",Coffee Shop,Italian Restaurant,Restaurant,Café,Bar,Bakery,Gym / Fitness Center,Sandwich Place,French Restaurant,Spa
3,Central Bay Street,Coffee Shop,Clothing Store,Middle Eastern Restaurant,Electronics Store,Chinese Restaurant,Japanese Restaurant,Sushi Restaurant,Tea Room,Plaza,Gym / Fitness Center
4,Christie,Café,Grocery Store,Athletics & Sports,Playground,Candy Store,Coffee Shop,Italian Restaurant,Baby Store,Fast Food Restaurant,Falafel Restaurant


## Cluster Neighborhoods

In [71]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 2, 2, 2, 2, 2, 1, 2], dtype=int32)

#### Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [82]:
toronto_merged = toronto_df

# add clustering labels
#toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood', lsuffix='_left')
#df_a.join(df_b, on='mukey', how='left', lsuffix='_left', rsuffix='_right')
#df_a.merge(df_b, on='mukey', how='left')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels_left,ClusterLabels,Cluster_Labels,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.650295,-79.359166,2,2,2,2,Coffee Shop,Bakery,Boat or Ferry,Theater,Gym Pool,Café,Gastropub,Spa,Dessert Shop,Shoe Store
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657363,-79.378180,2,2,2,2,Coffee Shop,Clothing Store,Japanese Restaurant,Middle Eastern Restaurant,Cosmetics Shop,Café,Bookstore,Fast Food Restaurant,Electronics Store,Italian Restaurant
2,M5C,Downtown Toronto,St. James Town,43.651210,-79.375481,2,2,2,2,Coffee Shop,Seafood Restaurant,Restaurant,Café,Hotel,Bakery,Diner,Cosmetics Shop,Italian Restaurant,Beer Bar
3,M4E,East Toronto,The Beaches,43.676531,-79.295425,2,2,2,2,Health Food Store,Pub,Trail,Wings Joint,Electronics Store,Flea Market,Fish Market,Fish & Chips Shop,Film Studio,Fast Food Restaurant
4,M5E,Downtown Toronto,Berczy Park,43.645160,-79.373675,2,2,2,2,Coffee Shop,Restaurant,Seafood Restaurant,Farmers Market,Café,Bakery,Hotel,Cocktail Bar,Beer Bar,Cheese Shop


### Visualize Clusters

In [83]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clulsters

#### Cluster 1

In [84]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels_left,ClusterLabels,Cluster_Labels,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,East YorkEast Toronto,2,0,0,0,Convenience Store,Film Studio,Farmers Market,Park,Italian Restaurant,Wings Joint,Empanada Restaurant,Flower Shop,Flea Market,Fish Market
10,Downtown Toronto,2,0,0,0,Harbor / Marina,Pier,Park,Thrift / Vintage Store,Wings Joint,Electronics Store,Fish Market,Fish & Chips Shop,Film Studio,Fast Food Restaurant
12,East Toronto,2,0,0,0,Bus Line,Discount Store,Ice Cream Shop,Grocery Store,Park,Ethiopian Restaurant,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop
22,West Toronto,0,0,0,0,Sandwich Place,Convenience Store,Park,Residential Building (Apartment / Condo),Wings Joint,Fish Market,Fish & Chips Shop,Film Studio,Fast Food Restaurant,Farmers Market
23,Central Toronto,2,0,0,0,Playground,Gym Pool,Park,Garden,Empanada Restaurant,Flea Market,Fish Market,Fish & Chips Shop,Film Studio,Fast Food Restaurant
33,Downtown Toronto,2,0,0,0,Playground,Candy Store,Grocery Store,Park,Empanada Restaurant,Flea Market,Fish Market,Fish & Chips Shop,Film Studio,Fast Food Restaurant


#### Cluster 2

In [85]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels_left,ClusterLabels,Cluster_Labels,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,Central Toronto,2,1,1,1,Food & Drink Shop,Gym,Japanese Restaurant,Park,Breakfast Spot,Hotel,Department Store,Flea Market,Fish Market,Fish & Chips Shop
29,Central Toronto,2,1,1,1,Convenience Store,Gym,Trail,Park,Wings Joint,Empanada Restaurant,Flea Market,Fish Market,Fish & Chips Shop,Film Studio


#### Cluster 3

In [86]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels_left,ClusterLabels,Cluster_Labels,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,2,2,2,2,Coffee Shop,Bakery,Boat or Ferry,Theater,Gym Pool,Café,Gastropub,Spa,Dessert Shop,Shoe Store
1,Downtown Toronto,2,2,2,2,Coffee Shop,Clothing Store,Japanese Restaurant,Middle Eastern Restaurant,Cosmetics Shop,Café,Bookstore,Fast Food Restaurant,Electronics Store,Italian Restaurant
2,Downtown Toronto,2,2,2,2,Coffee Shop,Seafood Restaurant,Restaurant,Café,Hotel,Bakery,Diner,Cosmetics Shop,Italian Restaurant,Beer Bar
3,East Toronto,2,2,2,2,Health Food Store,Pub,Trail,Wings Joint,Electronics Store,Flea Market,Fish Market,Fish & Chips Shop,Film Studio,Fast Food Restaurant
4,Downtown Toronto,2,2,2,2,Coffee Shop,Restaurant,Seafood Restaurant,Farmers Market,Café,Bakery,Hotel,Cocktail Bar,Beer Bar,Cheese Shop
5,Downtown Toronto,2,2,2,2,Coffee Shop,Clothing Store,Middle Eastern Restaurant,Electronics Store,Chinese Restaurant,Japanese Restaurant,Sushi Restaurant,Tea Room,Plaza,Gym / Fitness Center
6,Downtown Toronto,2,2,2,2,Café,Grocery Store,Athletics & Sports,Playground,Candy Store,Coffee Shop,Italian Restaurant,Baby Store,Fast Food Restaurant,Falafel Restaurant
7,Downtown Toronto,2,2,2,2,Coffee Shop,Café,Restaurant,Gastropub,Japanese Restaurant,Gym,Asian Restaurant,Steakhouse,Breakfast Spot,American Restaurant
8,West Toronto,1,2,2,2,Park,Bus Line,Middle Eastern Restaurant,Furniture / Home Store,Fast Food Restaurant,Smoke Shop,Pool,Bar,Bank,Bakery
11,West Toronto,2,2,2,2,Coffee Shop,Bar,Wine Bar,Asian Restaurant,Restaurant,Cocktail Bar,Vietnamese Restaurant,Pizza Place,Men's Store,Japanese Restaurant


#### Cluster 4

In [87]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels_left,ClusterLabels,Cluster_Labels,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Central Toronto,2,3,3,3,Health & Beauty Service,Wings Joint,Empanada Restaurant,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Film Studio,Fast Food Restaurant,Farmers Market


#### Cluster 5

In [88]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels_left,ClusterLabels,Cluster_Labels,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,Central Toronto,1,4,4,4,Park,Wings Joint,Empanada Restaurant,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Film Studio,Fast Food Restaurant,Farmers Market
